import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

In [6]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /home/akki/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [7]:
print(mnist_info)

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)



In [57]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [58]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [59]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image,label

In [60]:
train_val_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [67]:
buffer_size = 1000
train_val_data_shuffled = train_val_data.shuffle(buffer_size)
val_data = train_val_data_shuffled.take(num_validation_samples)
train_data = train_val_data_shuffled.skip(num_validation_samples)

batch_size = 150
train_data = train_data.batch(batch_size)
val_data = val_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

In [68]:
val_inputs, val_targets = next(iter(val_data))

In [69]:
input_size = 784
output_size = 10
model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28,28,1)),
            tf.keras.layers.Dense(200, activation='relu'),
            tf.keras.layers.Dense(200, activation='relu'),
            tf.keras.layers.Dense(200, activation='relu'),
            tf.keras.layers.Dense(200, activation='relu'),
            tf.keras.layers.Dense(output_size, activation='softmax')
])

In [70]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 200)               157000    
_________________________________________________________________
dense_39 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_40 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_41 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_42 (Dense)             (None, 10)                2010      
Total params: 279,610
Trainable params: 279,610
Non-trainable params: 0
________________________________________________

In [71]:
num_epochs = 5
model.fit(train_data, epochs=num_epochs, validation_data=(val_inputs, val_targets), verbose=2)

Epoch 1/5
54000/54000 - 334s - loss: 0.3531 - accuracy: 0.9103 - val_loss: 0.2140 - val_accuracy: 0.9522
Epoch 2/5
54000/54000 - 331s - loss: 0.2310 - accuracy: 0.9484 - val_loss: 0.1845 - val_accuracy: 0.9575
Epoch 3/5
54000/54000 - 331s - loss: 0.2092 - accuracy: 0.9562 - val_loss: 0.1663 - val_accuracy: 0.9643
Epoch 4/5
54000/54000 - 343s - loss: 0.1955 - accuracy: 0.9597 - val_loss: 0.3260 - val_accuracy: 0.9577
Epoch 5/5
54000/54000 - 490s - loss: 0.2065 - accuracy: 0.9621 - val_loss: 0.2720 - val_accuracy: 0.9572


In [72]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

InvalidArgumentError:  Matrix size-incompatible: In[0]: [1,7840000], In[1]: [784,200]
	 [[node sequential_9/dense_38/Relu (defined at <ipython-input-72-15516099b71f>:1) ]] [Op:__inference_test_function_614744]

Function call stack:
test_function
